In [13]:
# =============================================================
# 🎾 WINSTON LEE MATCH ANALYSIS — GENERALIZED SCRIPT
# =============================================================

import pandas as pd
import numpy as np
from IPython.display import display

# ===========================================================
# 🚀 SCRIPT CONFIGURATION
# ===========================================================
COURT_LENGTH_METERS = 23.77
PLAYER_NAME_RAW = "Winston Lee" # The player you are analyzing

# --- Load & standardize ---
try:
    df = pd.read_csv("/Users/viviana/Downloads/SwingVision-match-2025-10-18 at 20.55.46.xlsx - Shots.csv")
except FileNotFoundError:
    print("Error: Could not find the CSV file.")
    exit()
    
# Clean column names
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# --- DATA CLEANING ---

# Clean the 'type' column
if 'type' in df.columns:
    df['type'] = df['type'].str.strip().str.lower()

# --- NEW: Generalize Player Names ---
# This is the fix. We clean the player column and then
# convert all names that are NOT our player to the generic "opponent".

PLAYER_NAME = PLAYER_NAME_RAW.strip().lower()
OPPONENT_NAME = "opponent"

if 'player' in df.columns:
    # Clean whitespace and case
    df['player'] = df['player'].str.strip().str.lower()
    
    # Standardize names: "winston lee" stays, everyone else becomes "opponent"
    df['player'] = np.where(
        df['player'] == PLAYER_NAME,
        PLAYER_NAME,
        OPPONENT_NAME
    )
# --- END NEW FIX ---


# --- Helper functions ---
def add_rally_context(df_in):
    """
    Sorts by rally and adds 'next_' and 'prev_' columns for context.
    This is run ONCE to create the master analysis DataFrame.
    """
    df_sorted = df_in.sort_values(["set","game","point","shot"]).copy()
    
    # Columns to get context for
    context_cols = [
        "player", "type", "stroke", "hit_zone", 
        "direction", "bounce_depth", "result", "bounce_zone",
        "hit_(x)", "hit_(y)", "hit_(z)" 
    ]
    
    # Group by point to avoid data leaking across points
    g = df_sorted.groupby(["set","game","point"])

    # Add NEXT shot info (look forward)
    for col in context_cols:
        if col in df_sorted.columns:
            df_sorted[f"next_{col}"] = g[col].shift(-1)
            
    # Add PREVIOUS shot info (look backward)
    for col in context_cols:
        if col in df_sorted.columns:
            df_sorted[f"prev_{col}"] = g[col].shift(1)
            
    return df_sorted

def classify_serve_placement(direction):
    d = str(direction).lower()
    if "wide" in d: return "Wide"
    if "body" in d: return "Body"
    if "t" in d: return "T"
    return "Other"

def categorize_stroke(stroke):
    s = str(stroke).lower()
    if "forehand" in s: return "Forehand"
    if "backhand" in s: return "Backhand"
    if "volley" in s or "smash" in s or "overhead" in s: return "Volley"
    return None

def is_pressure_score(t_pts, o_pts):
    """
    Pressure = 30–30, any score where RECEIVER is within 1 point of game
    (e.g. 15–40, 30–40, 40–Ad), and any deuce.
    """
    # 30–30
    if (t_pts, o_pts) == (2, 2):
        return True

    # Break‑point type situations: opponent one point from game
    if o_pts >= 3 and t_pts <= o_pts - 1:
        return True
    if t_pts >= 3 and o_pts <= t_pts - 1:
        return True

    # Any deuce (40–40, 50–50, etc.)
    if t_pts >= 3 and o_pts >= 3 and t_pts == o_pts:
        return True

    return False

def section(title):
    print("\n" + "="*80)
    print(title)
    print("="*80)

# ===========================================================
# A) CORRECT point_winner & is_pressure
# ===========================================================

# 1) Sort in rally order (temporarily) to find point ends
df_sorted_temp = df.sort_values(["set","game","point","shot"])

# 2) Last shot of each point
point_end = (
    df_sorted_temp
    .groupby(["set","game","point"], as_index=False)
    .tail(1)
    .copy()
)

# 3) Infer point winner from FINAL shot
def infer_point_winner(row):
    player = row["player"]
    res = str(row["result"]).lower()
    if res in ["out", "net", "error"]:
        # hitter misses → opponent wins
        return OPPONENT_NAME if player == PLAYER_NAME else PLAYER_NAME
    else:
        # ball in / winner / etc. → hitter wins
        return player # This will be either PLAYER_NAME or OPPONENT_NAME

point_end["point_winner"] = point_end.apply(infer_point_winner, axis=1)

# 4) Merge point_winner back into ORIGINAL df
df = df.drop(columns=["point_winner"], errors="ignore")
df = df.merge(
    point_end[["set","game","point","point_winner"]],
    on=["set","game","point"],
    how="left"
)

# 5) Compute is_pressure by walking the game score
records = []
current_set, current_game = None, None
t_pts, o_pts = 0, 0

# Use the pre-calculated point_end DataFrame for this
for row in point_end.sort_values(["set","game","point"]).itertuples(index=False):
    s, g, p = row.set, row.game, row.point
    if (s, g) != (current_set, current_game):
        t_pts = o_pts = 0
        current_set, current_game = s, g
    
    pressure = is_pressure_score(t_pts, o_pts)
    records.append({"set": s, "game": g, "point": p, "is_pressure": pressure})
    
    if row.point_winner == PLAYER_NAME:
        t_pts += 1
    else:
        o_pts += 1

score_df = pd.DataFrame(records)
# Merge pressure info back into ORIGINAL df
df = df.merge(score_df, on=["set","game","point"], how="left")

# ===========================================================
# B) CREATE MASTER ANALYSIS DATAFRAME
# ===========================================================

# Run our enhanced helper function ONCE on the fully-enriched df
df_analysis = add_rally_context(df)

# Define court geometry based on config
BASELINE_A_Y = 0.0
BASELINE_B_Y = COURT_LENGTH_METERS
NET_CUTOFF_Y = COURT_LENGTH_METERS / 2

print("✅ Preprocessing complete. Master 'df_analysis' DataFrame is ready.")


# ===========================================================
# 1️⃣  1st Serve % on Pressure Points
# ===========================================================
section("1️⃣  1st Serve % on Pressure Points")

pressure_serves = df_analysis[
    (df_analysis["is_pressure"]) &
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"].isin(["first_serve","second_serve"]))
]

first_serves = pressure_serves[pressure_serves["type"] == "first_serve"]
first_in = first_serves[first_serves["result"].str.lower() == "in"]

pct1 = len(first_in) / len(first_serves) * 100 if len(first_serves) > 0 else np.nan
print(f"🎾 1st Serve % on pressure points = {pct1:.1f}% ({len(first_in)}/{len(first_serves)})")

# ===========================================================
# 2️⃣  Serves that generated short balls & placement
# ===========================================================
section("2️⃣  Serves that generated short balls & placement")

serves = df_analysis[
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"].isin(["first_serve","second_serve"]))
].copy() 

serves["serve_placement"] = serves["direction"].apply(classify_serve_placement)
serves["court_side"] = np.where(
    serves["bounce_zone"].str.contains("deuce", case=False, na=False),
    "Deuce",
    "Ad"
)
serves["placement_side"] = serves["court_side"] + " " + serves["serve_placement"]

serves_in = serves[
    (serves["result"].str.lower() == "in") &
    (serves["next_player"] == OPPONENT_NAME)
]

short_serves = serves_in[
    serves_in["next_bounce_depth"].str.lower() == "short"
]

pct_short = len(short_serves) / len(serves_in) * 100 if len(serves_in) > 0 else np.nan
print(f"Serves that generated short balls: {pct_short:.1f}% ({len(short_serves)}/{len(serves_in)})")

print("\nPlacement (all serves in):")
all_serves_placement_pct = serves_in["placement_side"].value_counts(normalize=True).mul(100)
all_serves_placement_count = serves_in["placement_side"].value_counts()
all_serves_df = pd.DataFrame({
    '%': all_serves_placement_pct.round(1), 
    'Count': all_serves_placement_count
})
display(all_serves_df)

print("\nPlacement (short-ball serves):")
short_serves_placement_pct = short_serves["placement_side"].value_counts(normalize=True).mul(100)
short_serves_placement_count = short_serves["placement_side"].value_counts()
short_serves_df = pd.DataFrame({
    '%': short_serves_placement_pct.round(1), 
    'Count': short_serves_placement_count
})
display(short_serves_df)

# ===========================================================
# 3️⃣  Winston's Returns Made % vs Opponent's 1st Serve (Pressure Points)
# ===========================================================
section("3️⃣  Winston's Returns Made % vs Opponent's 1st Serve (pressure points)")

winston_returns = df_analysis[
    (df_analysis["is_pressure"]) &
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"] == "first_return") &
    (df_analysis["prev_type"] == "first_serve") &
    (df_analysis["prev_player"] == OPPONENT_NAME)
]

winston_returns_in = winston_returns[
    winston_returns["result"].str.lower() == "in"
]

if len(winston_returns) > 0:
    winston_return_pct = len(winston_returns_in) / len(winston_returns) * 100
else:
    winston_return_pct = np.nan

print(f"Winston's Returns Made % vs Opponent's 1st Serve (pressure points): {winston_return_pct:.1f}% ({len(winston_returns_in)}/{len(winston_returns)})")

# ===========================================================
# 4️⃣  Point-Win % by Return Position (vs Opponent 1st Serve)
# ===========================================================
section("4️⃣  Point-Win % by Return Position (vs Opponent 1st Serve)")

winston_1st_returns = df_analysis[
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"] == "first_return") &
    (df_analysis["prev_player"] == OPPONENT_NAME) &
    (df_analysis["prev_type"] == "first_serve")
].copy() 

# Calculate relative depth based on which side of the net
side_a_mask = winston_1st_returns['hit_(y)'] <= NET_CUTOFF_Y
winston_1st_returns.loc[side_a_mask, 'hit_depth_relative'] = winston_1st_returns['hit_(y)'] - BASELINE_A_Y

side_b_mask = winston_1st_returns['hit_(y)'] > NET_CUTOFF_Y
winston_1st_returns.loc[side_b_mask, 'hit_depth_relative'] = BASELINE_B_Y - winston_1st_returns['hit_(y)']

# Now, classify "Deep" (negative) vs "Close" (positive)
winston_1st_returns["return_position"] = np.where(
    winston_1st_returns["hit_depth_relative"] < 0, 
    "Deep", 
    "Close"
)

winston_1st_returns["won"] = winston_1st_returns["point_winner"] == PLAYER_NAME

g = winston_1st_returns.groupby("return_position")["won"]
stat4_pct = g.mean().mul(100).round(1)
stat4_total = g.count()
stat4_won = g.sum()
stat4_df = pd.DataFrame({'Win %': stat4_pct, 'Won': stat4_won.astype(int), 'Total': stat4_total})
stat4_df['Ratio'] = stat4_df['Won'].astype(str) + '/' + stat4_df['Total'].astype(str)
display(stat4_df[['Win %', 'Ratio']])

# ===========================================================
# 5️⃣  Returns Made % vs Opponent's 2nd Serve (Pressure Points)
# ===========================================================
section("5️⃣  Returns Made % vs Opponent's 2nd Serve (pressure points)")

winston_returns_2nd = df_analysis[
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"] == "second_return") &
    (df_analysis["prev_player"] == OPPONENT_NAME) &
    (df_analysis["prev_type"] == "second_serve") &
    (df_analysis["is_pressure"])
]

total_returns = len(winston_returns_2nd)
returns_in = (winston_returns_2nd["result"].str.lower() == "in").sum()
returns_made_pct = (returns_in / total_returns * 100) if total_returns > 0 else np.nan

print(f"Winston's Returns Made % vs Opponent's 2nd Serve (pressure points): {returns_made_pct:.1f}% ({returns_in}/{total_returns})")

# ===========================================================
# 6️⃣  Point-Win % by Return Depth vs Opponent's 2nd Serve
# ===========================================================
section("6️⃣  Point-Win % by Return Depth vs Opponent's 2nd Serve")

winston_returns_2nd_all = df_analysis[
    (df_analysis["player"] == PLAYER_NAME) &
    (df_analysis["type"] == "second_return") &
    (df_analysis["prev_player"] == OPPONENT_NAME) &
    (df_analysis["prev_type"] == "second_serve")
].copy() 

# Calculate relative depth based on which side of the net
side_a_mask_s2 = winston_returns_2nd_all['hit_(y)'] <= NET_CUTOFF_Y
winston_returns_2nd_all.loc[side_a_mask_s2, 'hit_depth_relative'] = winston_returns_2nd_all['hit_(y)'] - BASELINE_A_Y

side_b_mask_s2 = winston_returns_2nd_all['hit_(y)'] > NET_CUTOFF_Y
winston_returns_2nd_all.loc[side_b_mask_s2, 'hit_depth_relative'] = BASELINE_B_Y - winston_returns_2nd_all['hit_(y)']

# Now, classify "Deep" (negative) vs "Close" (positive)
winston_returns_2nd_all["return_position"] = np.where(
    winston_returns_2nd_all["hit_depth_relative"] < 0, 
    "Deep", 
    "Close"
)

winston_returns_2nd_all["won"] = winston_returns_2nd_all["point_winner"] == PLAYER_NAME

g = winston_returns_2nd_all.groupby("return_position")["won"]
stat6_pct = g.mean().mul(100).round(1)
stat6_total = g.count()
stat6_won = g.sum()
stat6_df = pd.DataFrame({'Win %': stat6_pct, 'Won': stat6_won.astype(int), 'Total': stat6_total})
stat6_df['Ratio'] = stat6_df['Won'].astype(str) + '/' + stat6_df['Total'].astype(str)
display(stat6_df[['Win %', 'Ratio']])

# ===========================================================
# 7️⃣  Winners & Forced Errors % by Stroke on Pressure Points
# ===========================================================
section("7️⃣  Winners & Forced Errors % by Stroke on Pressure Points")

df_pressure = df_analysis[
    (df_analysis["is_pressure"]) &
    (df_analysis["player"] == PLAYER_NAME)
].copy() 

df_pressure["is_last_shot"] = pd.isna(df_pressure["next_player"])
df_pressure["stroke_group"] = df_pressure["stroke"].apply(categorize_stroke)

def classify_outcome(row):
    winston_wins = row["point_winner"] == PLAYER_NAME
    
    if row["is_last_shot"] and str(row["result"]).lower() == "in" and winston_wins:
        return "Winner"
        
    if (
        row["next_player"] == OPPONENT_NAME and
        str(row["next_result"]).lower() in ["out","net","error"] and
        winston_wins
    ):
        return "Forced Error"
    return "Other"

df_pressure["outcome"] = df_pressure.apply(classify_outcome, axis=1)

agg = df_pressure.groupby("stroke_group")["outcome"].value_counts().unstack(fill_value=0)

for c in ["Winner","Forced Error", "Other"]:
    if c not in agg.columns:
        agg[c] = 0

agg["Total"] = agg.sum(axis=1)
agg["Winner %"] = (agg["Winner"] / agg["Total"] * 100).round(1)
agg["Forced %"] = (agg["Forced Error"] / agg["Total"] * 100).round(1)
agg["Winner+Forced %"] = ((agg["Winner"] + agg["Forced Error"]) / agg["Total"] * 100).round(1)

display_cols = [
    "Total", 
    "Winner", "Winner %", 
    "Forced Error", "Forced %", 
    "Winner+Forced %"
]
display_cols = [c for c in display_cols if c in agg.columns]
display(agg[display_cols])

# ===========================================================
# 8️⃣  Unforced Error % by Stroke on Pressure Points
# ===========================================================
section("8️⃣  Unforced Error % by Stroke on Pressure Points")

df_unf = df_analysis[
    (df_analysis["is_pressure"]) &
    (df_analysis["player"] == PLAYER_NAME)
].copy() 

df_unf["stroke_group"] = df_unf["stroke"].apply(categorize_stroke)
df_unf = df_unf.dropna(subset=["stroke_group"])
df_unf["is_unforced_error"] = df_unf["result"].str.lower().isin(["out","net","error"])

g8 = df_unf.groupby("stroke_group")["is_unforced_error"]
summary_pct = g8.mean().mul(100).round(1)
summary_total = g8.count()
summary_errors = g8.sum()

summary_df = pd.DataFrame({
    'Unforced Error %': summary_pct, 
    'Errors': summary_errors.astype(int), 
    'Total Shots': summary_total
})
summary_df['Ratio'] = summary_df['Errors'].astype(str) + '/' + summary_df['Total Shots'].astype(str)
display(summary_df[['Unforced Error %', 'Ratio']])

# ===========================================================
# 9️⃣  Net Points Won % on Pressure Points
# ===========================================================
section("9️⃣  Net Points Won % on Pressure Points")

df_net = df_analysis[
    (df_analysis["is_pressure"]) &
    (df_analysis["player"] == PLAYER_NAME)
].copy() 

def is_net_play(stroke, zone):
    s, z = str(stroke).lower(), str(zone).lower()
    return ("volley" in s or "smash" in s or "overhead" in s or "short" in z or "net" in z)

df_net["net_approach"] = df_net.apply(
    lambda r: is_net_play(r["stroke"], r["hit_zone"]),
    axis=1
)

pts_net = df_net[df_net["net_approach"]].groupby(
    ["set","game","point"], as_index=False
).agg({"point_winner": "first"}) 

pct9 = (pts_net["point_winner"] == PLAYER_NAME).mean() * 100 if len(pts_net) > 0 else np.nan
print(f"Net Points Won % = {pct9:.1f}% ({(pts_net['point_winner']==PLAYER_NAME).sum()}/{len(pts_net)})")

# ===========================================================
# 🔟  Opportunities to Attack / Come In (All Points)
# ===========================================================
section("🔟  Opportunities to Attack / Come In (All Points)")

# We now ONLY exclude the serves, returns, and feeds.
# 'serve_plus_one' and 'return_plus_one' are now ALLOWED as opportunities.
rally_exclude = {"first_serve","second_serve","first_return","second_return","feed"}

short_opps = df_analysis[
    (df_analysis["player"] == OPPONENT_NAME) &
    (df_analysis["result"].str.lower() == "in") &
    (df_analysis["bounce_depth"].str.lower() == "short") &
    (~df_analysis["type"].str.lower().isin(rally_exclude)) &
    (df_analysis["next_player"] == PLAYER_NAME) &
    (df_analysis["next_stroke"].notna())
].copy() 

def took_opp(row):
    s = str(row["next_stroke"]).lower()
    z = str(row["next_hit_zone"]).lower()
    d = str(row["next_direction"]).lower()
    return (
        "volley" in s or
        "smash" in s or
        "overhead" in s or
        "short" in z or
        "net" in z or
        ("line" in d or "inside" in d) 
    )

short_opps["took_opportunity"] = short_opps.apply(took_opp, axis=1)
short_opps["tiago_won"] = short_opps["point_winner"] == PLAYER_NAME

tot = len(short_opps)
took = int(short_opps["took_opportunity"].sum()) 
miss = tot - took
succ = int(short_opps.loc[short_opps["took_opportunity"], "tiago_won"].sum())
succ_pct = (succ / took * 100) if took > 0 else np.nan

print(f"Total short-ball opps: {tot}, Taken: {took}, Missed: {miss}, Success when taken: {succ_pct:.1f}% ({succ}/{took})")

print("\n=== SCRIPT FINISHED ===")

✅ Preprocessing complete. Master 'df_analysis' DataFrame is ready.

1️⃣  1st Serve % on Pressure Points
🎾 1st Serve % on pressure points = nan% (0/0)

2️⃣  Serves that generated short balls & placement
Serves that generated short balls: nan% (0/0)

Placement (all serves in):


,%,Count
placement_side,,



Placement (short-ball serves):


,%,Count
placement_side,,



3️⃣  Winston's Returns Made % vs Opponent's 1st Serve (pressure points)
Winston's Returns Made % vs Opponent's 1st Serve (pressure points): nan% (0/0)

4️⃣  Point-Win % by Return Position (vs Opponent 1st Serve)


...


7️⃣  Winners & Forced Errors % by Stroke on Pressure Points


outcome,Total,Winner,Winner %,Forced Error,Forced %,Winner+Forced %
stroke_group,,,,,,
Backhand,82,5,6.1,11,13.4,19.5
Forehand,98,6,6.1,7,7.1,13.3
Volley,13,2,15.4,1,7.7,23.1



8️⃣  Unforced Error % by Stroke on Pressure Points


,Unforced Error %,Ratio
stroke_group,,
Backhand,13.4,11/82
Forehand,18.4,18/98
Volley,53.8,7/13



9️⃣  Net Points Won % on Pressure Points
Net Points Won % = 36.4% (4/11)

🔟  Opportunities to Attack / Come In (All Points)
Total short-ball opps: 84, Taken: 67, Missed: 17, Success when taken: 38.8% (26/67)

=== SCRIPT FINISHED ===


In [15]:
# ===========================================================
# 🚀 GOOGLE SHEETS - SINGLE-LINE OUTPUT
# ===========================================================

# This section collects all the stats you've already calculated
# and formats them into a single, comma-separated line for Google Sheets.

try:
    csv_output_list = []

    # --- Q1: 1st Serve % (Pressure) ---
    q1_str = f"{pct1:.1f}% ({len(first_in)}/{len(first_serves)})" if not np.isnan(pct1) else f"nan% ({len(first_in)}/{len(first_serves)})"
    csv_output_list.append(q1_str)

    # --- Q2: Short Ball % (Serve) ---
    q2_str = f"{pct_short:.1f}% ({len(short_serves)}/{len(serves_in)})" if not np.isnan(pct_short) else f"nan% ({len(short_serves)}/{len(serves_in)})"
    csv_output_list.append(q2_str)

    # --- Q3: 1st Serve Return % (Pressure) ---
    q3_str = f"{winston_return_pct:.1f}% ({len(winston_returns_in)}/{len(winston_returns)})" if not np.isnan(winston_return_pct) else f"nan% ({len(winston_returns_in)}/{len(winston_returns)})"
    csv_output_list.append(q3_str)

    # --- Q4: Win % vs 1st Srv ---
    q4_close_str = "nan% (0/0)"
    if 'Close' in stat4_df.index:
        q4_close_str = f"{stat4_df.loc['Close', 'Win %']}% ({stat4_df.loc['Close', 'Ratio']})"
    csv_output_list.append(q4_close_str)
    
    q4_deep_str = "nan% (0/0)"
    if 'Deep' in stat4_df.index:
        q4_deep_str = f"{stat4_df.loc['Deep', 'Win %']}% ({stat4_df.loc['Deep', 'Ratio']})"
    csv_output_list.append(q4_deep_str)

    # --- Q5: 2nd Serve Return % (Pressure) ---
    q5_str = f"{returns_made_pct:.1f}% ({returns_in}/{total_returns})" if not np.isnan(returns_made_pct) else f"nan% ({returns_in}/{total_returns})"
    csv_output_list.append(q5_str)

    # --- Q6: Win % vs 2nd Srv ---
    q6_close_str = "nan% (0/0)"
    if 'Close' in stat6_df.index:
        q6_close_str = f"{stat6_df.loc['Close', 'Win %']}% ({stat6_df.loc['Close', 'Ratio']})"
    csv_output_list.append(q6_close_str)
    
    q6_deep_str = "nan% (0/0)"
    if 'Deep' in stat6_df.index:
        q6_deep_str = f"{stat6_df.loc['Deep', 'Win %']}% ({stat6_df.loc['Deep', 'Ratio']})"
    csv_output_list.append(q6_deep_str)

    # --- Q7: Winners+Forced (Pressure) [MODIFIED] ---
    q7_fh_str = "nan% (0/0)"
    if 'Forehand' in agg.index:
        combined_pct = agg.loc['Forehand', 'Winner+Forced %']
        combined_count = agg.loc['Forehand', 'Winner'] + agg.loc['Forehand', 'Forced Error']
        total_count = agg.loc['Forehand', 'Total']
        q7_fh_str = f"{combined_pct}% ({combined_count}/{total_count})"
    csv_output_list.append(q7_fh_str)

    q7_bh_str = "nan% (0/0)"
    if 'Backhand' in agg.index:
        combined_pct = agg.loc['Backhand', 'Winner+Forced %']
        combined_count = agg.loc['Backhand', 'Winner'] + agg.loc['Backhand', 'Forced Error']
        total_count = agg.loc['Backhand', 'Total']
        q7_bh_str = f"{combined_pct}% ({combined_count}/{total_count})"
    csv_output_list.append(q7_bh_str)
    # --- END MODIFIED ---

    # --- Q8: Unforced Error % (Pressure) ---
    q8_fh_str = "nan% (0/0)"
    if 'Forehand' in summary_df.index:
        q8_fh_str = f"{summary_df.loc['Forehand', 'Unforced Error %']}% ({summary_df.loc['Forehand', 'Ratio']})"
    csv_output_list.append(q8_fh_str)

    q8_bh_str = "nan% (0/0)"
    if 'Backhand' in summary_df.index:
        q8_bh_str = f"{summary_df.loc['Backhand', 'Unforced Error %']}% ({summary_df.loc['Backhand', 'Ratio']})"
    csv_output_list.append(q8_bh_str)
    
    # --- Q9: Net Point Win % (Pressure) ---
    q9_won = (pts_net['point_winner']==PLAYER_NAME).sum()
    q9_total = len(pts_net)
    q9_str = f"{pct9:.1f}% ({q9_won}/{q9_total})" if not np.isnan(pct9) else f"nan% ({q9_won}/{q9_total})"
    csv_output_list.append(q9_str)
    
    # --- Q10: Opps ---
    csv_output_list.append(str(tot))
    csv_output_list.append(str(took))
    csv_output_list.append(str(miss))
    q10_str = f"{succ_pct:.1f}% ({succ}/{took})" if not np.isnan(succ_pct) else f"nan% ({succ}/{took})"
    csv_output_list.append(q10_str)
    
    # --- Final Print ---
    final_output_line = ",".join(csv_output_list)
    
    print("\n" + "="*80)
    print("✅ GOOGLE SHEETS COPY-PASTE LINE:")
    print("="*80)
    print(final_output_line)

except Exception as e:
    print(f"\n--- ERROR generating Google Sheets line: {e} ---")
    print("This likely happened because a stat (e.g., 'stat4_df') was empty.")


✅ GOOGLE SHEETS COPY-PASTE LINE:
nan% (0/0),nan% (0/0),nan% (0/0),nan% (0/0),nan% (0/0),nan% (0/0),nan% (0/0),nan% (0/0),13.3% (13/98),19.5% (16/82),18.4% (18/98),13.4% (11/82),36.4% (4/11),84,67,17,38.8% (26/67)
